In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

In [2]:
raw_data = pd.read_excel('../data/ISI,ESS,PSGdata_20220113.xlsx')
raw_data.head()

/var/folders/l6/j5nwgjg11zqbq0y11b2myyb00000gn/T/ipykernel_12517/2156600217.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  raw_data = pd.read_excel('../data/ISI,ESS,PSGdata_20220113.xlsx')


,ID,PSG_date,PSG_ID,sex,age,birth,Final Diagnosis,OSA,insomnia,"PLMD, RLS",...,PVC(y/n),Bradycardia(y/n),Others(y/n),PLM#,Total \nLMI =PLMI(/h),Total LM \nArousal#,Total LM AI(/h),PLM \nArousal#,PLM \nAI(/h),MAI(/h)
0,1,2014-12-10,N14005,1,52,1962-02-26,sleep talking,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2014-12-10,PE141015,0,65,1948-12-17,RLS/OSA,1,0,1,...,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN,NaN
2,3,2014-12-10,PE141017,0,53,1961-02-28,OSA,1,0,0,...,NaN,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN
3,4,2014-12-10,PE141014,0,53,1961-06-01,OSA/PPI,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-03-31,PE150253,1,55,1959-12-19,OSA,1,0,0,...,N,N,N,22.0,3.2,2.0,0.3,0.0,0.0,0.3


In [3]:
raw_data.shape

(5481, 140)

In [5]:
raw_data.iloc[:, 0:15]

,ID,PSG_date,PSG_ID,sex,age,birth,Final Diagnosis,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,1,2014-12-10,N14005,1,52,1962-02-26,sleep talking,0,0,0,0,0,0,0,1
1,2,2014-12-10,PE141015,0,65,1948-12-17,RLS/OSA,1,0,1,0,0,0,0,0
2,3,2014-12-10,PE141017,0,53,1961-02-28,OSA,1,0,0,0,0,0,0,0
3,4,2014-12-10,PE141014,0,53,1961-06-01,OSA/PPI,1,1,0,0,0,0,0,0
4,5,2015-03-31,PE150253,1,55,1959-12-19,OSA,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,5477,2019-08-06,PE190964,1,53,1966-06-30,OSA,1,0,0,0,0,0,0,0
5477,5478,2020-04-23,PE200452,1,67,1952-09-21,OSA,1,0,0,0,0,0,0,0
5478,5479,2020-06-30,PE200722,1,54,1965-07-28,PPI,0,1,0,0,0,0,0,0
5479,5480,2021-04-29,PE210482,1,63,1958-02-18,OSA,1,0,0,0,0,0,0,0


In [11]:
raw_data.iloc[:, 15:23]

,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,999,999,999,999,999,999,999,999
...,...,...,...,...,...,...,...,...
5476,3,3,3,3,3,3,3,3
5477,3,3,3,3,3,3,3,3
5478,3,3,3,3,3,3,3,3
5479,3,3,3,3,3,3,3,3


In [15]:
raw_data.iloc[:, 23:30]

,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5
0,1,0,0,0.0,4.0,4,4
1,4,4,4,4.0,1.0,2,4
2,0,1,0,2.0,2.0,2,2
3,3,2,2,3.0,3.0,3,3
4,4,4,4,4.0,4.0,2,4
...,...,...,...,...,...,...,...
5476,4,4,4,4.0,3.0,3,4
5477,0,0,0,1.0,0.0,4,4
5478,0,0,0,4.0,3.0,4,3
5479,0,0,0,2.0,3.0,4,0


In [17]:
raw_data.iloc[:, 30:39]

,SSQ1,SSQ2,SSQ3,SSQ4,SSQ5,SSQ6,SSQ7,SSQ8,SSQ9
0,999,999,999,999,999,999,999,999,999
1,999,999,999,999,999,999,999,999,999
2,999,999,999,999,999,999,999,999,999
3,999,999,999,999,999,999,999,999,999
4,999,999,999,999,999,999,999,999,999
...,...,...,...,...,...,...,...,...,...
5476,1,1,1,1,1,0,0,0,1
5477,1,1,1,0,1,1,0,0,1
5478,1,0,1,0,1,0,0,1,0
5479,1,1,1,0,0,0,0,1,0


In [26]:
raw_data.iloc[:, 39:60]

,신장,체중,BMI,PSQI,ISI,SSS,ESS,HADS-A,HADS-D,K-BDI2,...,배둘레,엉덩이둘레,목둘레\n앉은상태,목둘레\n누운상태,머리둘레,전날수면시간,검사당일수면제\n복용여부(Y/N),수면제이름,검사당일\n음주여부(Y/N),검사당일\n음주량
0,167.0,60.0,21.51,999,999.0,999,999,999,999,999,...,999.0,999.0,999.0,999.0,999.0,0,NaN,NaN,NaN,NaN
1,166.0,53.0,19.23,999,999.0,999,999,999,999,999,...,999.0,999.0,999.0,999.0,999.0,0,NaN,NaN,NaN,NaN
2,174.0,74.0,24.44,999,999.0,999,999,999,999,999,...,999.0,999.0,999.0,999.0,999.0,0,NaN,NaN,NaN,NaN
3,163.0,63.5,23.90,999,999.0,999,999,999,999,999,...,999.0,999.0,999.0,999.0,999.0,0,NaN,NaN,NaN,NaN
4,150.0,61.0,27.11,999,999.0,999,999,999,999,999,...,999.0,999.0,999.0,999.0,999.0,999,N,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,160.0,47.0,18.40,18,26.0,6,24,999,999,58,...,82.0,85.0,31.0,31.5,51.5,5,Y,"diaz 5mg, triaz 0.25mg, Escitalo 5mg",N,NaN
5477,145.0,65.0,30.90,0,9.0,7,24,999,999,999,...,108.0,93.0,38.0,39.0,56.0,0,N,NaN,N,NaN
5478,154.0,76.0,32.00,7,14.0,6,24,999,999,40,...,98.0,104.0,36.0,36.5,55.0,11,N,NaN,N,NaN
5479,154.0,50.0,21.10,6,9.0,3,24,999,999,10,...,999.0,999.0,999.0,999.0,999.0,7,Y,NaN,N,NaN


In [30]:
targets = raw_data.iloc[:, 7:15].copy()
targets.head()

,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


# ISI

In [31]:
isi_qs = raw_data.iloc[:, 23:30].copy()
isi_qs.head()

,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5
0,1,0,0,0.0,4.0,4,4
1,4,4,4,4.0,1.0,2,4
2,0,1,0,2.0,2.0,2,2
3,3,2,2,3.0,3.0,3,3
4,4,4,4,4.0,4.0,2,4


In [32]:
isi_data = pd.concat([isi_qs, targets], axis=1)
isi_data.head()

,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,1,0,0,0.0,4.0,4,4,0,0,0,0,0,0,0,1
1,4,4,4,4.0,1.0,2,4,1,0,1,0,0,0,0,0
2,0,1,0,2.0,2.0,2,2,1,0,0,0,0,0,0,0
3,3,2,2,3.0,3.0,3,3,1,1,0,0,0,0,0,0
4,4,4,4,4.0,4.0,2,4,1,0,0,0,0,0,0,0


In [33]:
isi_data.to_csv('../data/isi_only.csv', index=False)

In [34]:
print(isi_data.shape)

isi_missings = isi_data.isnull().sum()
isi_missings

(5481, 15)


ISI1a                                   0
ISI1b                                   0
ISI1c                                   0
ISI2                                    0
ISI3                                    0
ISI4                                    0
ISI5                                    0
OSA                                     0
insomnia                                0
PLMD, RLS                               0
parasomnia                              0
circadian rhythm sleep-wake disorder    0
Narcolepsy, EDS, Hypersomnia            0
Catathrenia                             0
control                                 0
dtype: int64

In [37]:
isi_data.dtypes

ISI1a                                     int64
ISI1b                                     int64
ISI1c                                     int64
ISI2                                    float64
ISI3                                    float64
ISI4                                      int64
ISI5                                      int64
OSA                                       int64
insomnia                                  int64
PLMD, RLS                                 int64
parasomnia                                int64
circadian rhythm sleep-wake disorder      int64
Narcolepsy, EDS, Hypersomnia              int64
Catathrenia                               int64
control                                   int64
dtype: object

In [36]:
na_cols_list = []
for col in isi_data.columns:
    na_row = isi_data[isi_data[col] == 999]
    if len(na_row) != 0:
        na_cols_list.append(col)
        print(col, '\t', len(na_row))

OSA 	 1
insomnia 	 1
Catathrenia 	 1
control 	 1


In [39]:
for col in na_cols_list:
    print(isi_data[isi_data[col] == 999].index)

Int64Index([3570], dtype='int64')
Int64Index([3570], dtype='int64')
Int64Index([3570], dtype='int64')
Int64Index([3570], dtype='int64')


In [40]:
isi_data = isi_data.drop([3570], axis=0)

for col in isi_data.columns:
    na_row = isi_data[isi_data[col] == 999]
    if len(na_row) != 0:
        print(col, '\t', len(na_row))

In [42]:
isi_data[['ISI2', 'ISI3']] = isi_data[['ISI2', 'ISI3']].astype(int)

In [43]:
isi_data.dtypes

ISI1a                                   int64
ISI1b                                   int64
ISI1c                                   int64
ISI2                                    int64
ISI3                                    int64
ISI4                                    int64
ISI5                                    int64
OSA                                     int64
insomnia                                int64
PLMD, RLS                               int64
parasomnia                              int64
circadian rhythm sleep-wake disorder    int64
Narcolepsy, EDS, Hypersomnia            int64
Catathrenia                             int64
control                                 int64
dtype: object

In [44]:
isi_data.to_csv('../data/isi_only_preprocessed.csv', index=False)

# ESS

In [45]:
ess_qs = raw_data.iloc[:, 15:23].copy()
ess_qs.head()

,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,999,999,999,999,999,999,999,999


In [46]:
ess_data = pd.concat([ess_qs, targets], axis=1)
ess_data.head()

,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,999,999,999,999,999,999,999,999,1,0,0,0,0,0,0,0


In [47]:
ess_data.to_csv('../data/ess_only.csv', index=False)
print(ess_data.shape)

ess_missings = ess_data.isnull().sum()
ess_missings

(5481, 16)


ESS1                                    0
ESS2                                    0
ESS3                                    0
ESS4                                    0
ESS5                                    0
ESS6                                    0
ESS7                                    0
ESS8                                    0
OSA                                     0
insomnia                                0
PLMD, RLS                               0
parasomnia                              0
circadian rhythm sleep-wake disorder    0
Narcolepsy, EDS, Hypersomnia            0
Catathrenia                             0
control                                 0
dtype: int64

In [48]:
ess_data.dtypes

ESS1                                    int64
ESS2                                    int64
ESS3                                    int64
ESS4                                    int64
ESS5                                    int64
ESS6                                    int64
ESS7                                    int64
ESS8                                    int64
OSA                                     int64
insomnia                                int64
PLMD, RLS                               int64
parasomnia                              int64
circadian rhythm sleep-wake disorder    int64
Narcolepsy, EDS, Hypersomnia            int64
Catathrenia                             int64
control                                 int64
dtype: object

In [49]:
na_cols_list = []
for col in ess_data.columns:
    na_row = ess_data[ess_data[col] == 999]
    if len(na_row) != 0:
        na_cols_list.append(col)
        print(col, '\t', len(na_row))

ESS1 	 17
ESS2 	 17
ESS3 	 17
ESS4 	 17
ESS5 	 17
ESS6 	 17
ESS7 	 17
ESS8 	 17
OSA 	 1
insomnia 	 1
Catathrenia 	 1
control 	 1


In [50]:
for col in na_cols_list:
    print(ess_data[ess_data[col] == 999].index)

Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935,
            3248],
           dtype='int64')
Int64Index([3570], dtype='int64'

In [52]:
ess_data = ess_data.drop([4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1924, 1934, 1935, 3248, 3570], axis=0)

for col in ess_data.columns:
    na_row = ess_data[ess_data[col] == 999]
    if len(na_row) != 0:
        print(col, '\t', len(na_row))

In [53]:
ess_data.to_csv('../data/ess_only_preprocessed.csv', index=False)

# SSQ

In [54]:
ssq_qs = raw_data.iloc[:, 30:39].copy()
ssq_qs.head()

,SSQ1,SSQ2,SSQ3,SSQ4,SSQ5,SSQ6,SSQ7,SSQ8,SSQ9
0,999,999,999,999,999,999,999,999,999
1,999,999,999,999,999,999,999,999,999
2,999,999,999,999,999,999,999,999,999
3,999,999,999,999,999,999,999,999,999
4,999,999,999,999,999,999,999,999,999


In [55]:
ssq_data = pd.concat([ssq_qs, targets], axis=1)
ssq_data.head()

,SSQ1,SSQ2,SSQ3,SSQ4,SSQ5,SSQ6,SSQ7,SSQ8,SSQ9,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,999,999,999,999,999,999,999,999,999,0,0,0,0,0,0,0,1
1,999,999,999,999,999,999,999,999,999,1,0,1,0,0,0,0,0
2,999,999,999,999,999,999,999,999,999,1,0,0,0,0,0,0,0
3,999,999,999,999,999,999,999,999,999,1,1,0,0,0,0,0,0
4,999,999,999,999,999,999,999,999,999,1,0,0,0,0,0,0,0


In [56]:
ssq_data.to_csv('../data/ssq_only.csv', index=False)
print(ssq_data.shape)

ssq_missings = ssq_data.isnull().sum()
ssq_missings

(5481, 17)


SSQ1                                    0
SSQ2                                    0
SSQ3                                    0
SSQ4                                    0
SSQ5                                    0
SSQ6                                    0
SSQ7                                    0
SSQ8                                    0
SSQ9                                    0
OSA                                     0
insomnia                                0
PLMD, RLS                               0
parasomnia                              0
circadian rhythm sleep-wake disorder    0
Narcolepsy, EDS, Hypersomnia            0
Catathrenia                             0
control                                 0
dtype: int64

In [57]:
ssq_data.dtypes

SSQ1                                     int64
SSQ2                                    object
SSQ3                                     int64
SSQ4                                     int64
SSQ5                                     int64
SSQ6                                     int64
SSQ7                                     int64
SSQ8                                     int64
SSQ9                                     int64
OSA                                      int64
insomnia                                 int64
PLMD, RLS                                int64
parasomnia                               int64
circadian rhythm sleep-wake disorder     int64
Narcolepsy, EDS, Hypersomnia             int64
Catathrenia                              int64
control                                  int64
dtype: object

In [58]:
print(ssq_data[(ssq_data['SSQ2'] == '-') | (ssq_data['SSQ2'] == ' ')].index)

Int64Index([1377], dtype='int64')


In [59]:
ssq_data = ssq_data.drop([1377], axis=0)
ssq_data['SSQ2'] = ssq_data['SSQ2'].astype(int)

In [60]:
ssq_data.dtypes

SSQ1                                    int64
SSQ2                                    int64
SSQ3                                    int64
SSQ4                                    int64
SSQ5                                    int64
SSQ6                                    int64
SSQ7                                    int64
SSQ8                                    int64
SSQ9                                    int64
OSA                                     int64
insomnia                                int64
PLMD, RLS                               int64
parasomnia                              int64
circadian rhythm sleep-wake disorder    int64
Narcolepsy, EDS, Hypersomnia            int64
Catathrenia                             int64
control                                 int64
dtype: object

In [62]:
na_cols_list = []
for col in ssq_data.columns:
    na_row = ssq_data[ssq_data[col] == 999]
    if len(na_row) != 0:
        na_cols_list.append(col)
        print(col, '\t', len(na_row))

SSQ1 	 3083
SSQ2 	 3116
SSQ3 	 3074
SSQ4 	 3080
SSQ5 	 3084
SSQ6 	 3127
SSQ7 	 3125
SSQ8 	 3087
SSQ9 	 3076
OSA 	 1
insomnia 	 1
Catathrenia 	 1
control 	 1


In [63]:
na_indices = []
for col in na_cols_list:
    na_indices += list(ssq_data[ssq_data[col] == 999].index)

na_indices = list(set(na_indices))
na_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 78,
 82,
 84,
 88,
 90,
 94,
 99,
 102,
 103,
 106,
 108,
 112,
 114,
 116,
 119,
 123,
 125,
 126,
 129,
 130,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 2

In [64]:
ssq_data = ssq_data.drop(na_indices, axis=0)

for col in ssq_data.columns:
    na_row = ssq_data[ssq_data[col] == 999]
    if len(na_row) != 0:
        print(col, '\t', len(na_row))

In [65]:
ssq_data.to_csv('../data/ssq_only_preprocessed.csv', index=False)

# split data

In [66]:
from sklearn.model_selection import train_test_split

In [69]:
target_isi = isi_data.iloc[:, 7:15]
features_isi = isi_data.drop(isi_data.iloc[:, 7:15], axis=1)

X_train_isi, X_test_isi, y_train_isi, y_test_isi = train_test_split(features_isi, target_isi, test_size=0.2)

In [71]:
train_isi = pd.concat([X_train_isi, y_train_isi], axis=1)
train_isi.head()

,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
4767,1,1,0,2,3,3,3,0,0,0,0,0,0,0,0
4510,0,3,3,3,2,2,2,1,1,1,0,0,0,0,0
1123,2,2,2,2,2,0,3,0,1,0,0,0,0,0,0
3157,2,2,2,3,2,1,3,1,0,0,0,0,0,0,0
3509,1,1,1,3,2,2,3,1,0,0,0,0,0,0,0


In [72]:
test_isi = pd.concat([X_test_isi, y_test_isi], axis=1)
test_isi.head()

,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
5470,0,0,0,4,4,4,4,0,0,1,0,0,0,0,0
3069,0,0,1,2,1,0,0,1,0,0,0,0,0,0,0
5145,1,1,2,3,2,2,3,1,0,0,0,0,0,0,0
2995,2,2,2,2,2,0,1,0,0,0,1,0,0,0,0
5225,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0


In [73]:
train_isi.to_csv('../data/isi_only_train.csv', index=False)
test_isi.to_csv('../data/isi_only_test.csv', index=False)

In [76]:
target_ess = ess_data.iloc[:, 8:16]
features_ess = ess_data.drop(ess_data.iloc[:, 8:16], axis=1)

X_train_ess, X_test_ess, y_train_ess, y_test_ess = train_test_split(features_ess, target_ess, test_size=0.2)

In [77]:
train_ess = pd.concat([X_train_ess, y_train_ess], axis=1)
train_ess.head()

,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
1871,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0
4598,3,2,1,3,3,0,1,2,1,0,0,0,0,0,0,0
3683,2,1,2,3,1,0,1,2,0,1,0,0,0,0,0,0
3059,2,1,2,1,1,1,1,1,1,0,0,0,0,0,0,0
3854,2,1,1,3,1,0,1,3,1,0,0,0,0,0,0,0


In [78]:
test_ess = pd.concat([X_test_ess, y_test_ess], axis=1)
test_ess.head()

,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
4382,3,3,1,3,0,1,3,0,1,0,0,0,0,0,0,0
3338,2,1,0,1,2,2,2,1,1,0,0,0,0,0,0,0
2853,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1081,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0
140,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [79]:
train_ess.to_csv('../data/ess_only_train.csv', index=False)
test_ess.to_csv('../data/ess_only_test.csv', index=False)

In [82]:
target_ssq = ssq_data.iloc[:, 9:17]
features_ssq = ssq_data.drop(ssq_data.iloc[:, 9:17], axis=1)

X_train_ssq, X_test_ssq, y_train_ssq, y_test_ssq = train_test_split(features_ssq, target_ssq, test_size=0.2)

In [83]:
train_ssq = pd.concat([X_train_ssq, y_train_ssq], axis=1)
train_ssq.head()

,SSQ1,SSQ2,SSQ3,SSQ4,SSQ5,SSQ6,SSQ7,SSQ8,SSQ9,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
3145,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0
3064,0,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0
4170,1,0,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0
1878,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0
1063,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0


In [84]:
test_ssq = pd.concat([X_test_ssq, y_test_ssq], axis=1)
test_ssq.head()

,SSQ1,SSQ2,SSQ3,SSQ4,SSQ5,SSQ6,SSQ7,SSQ8,SSQ9,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
4156,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
5170,1,1,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0
2719,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0
2180,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3094,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0


In [85]:
train_ssq.to_csv('../data/ssq_only_train.csv', index=False)
test_ssq.to_csv('../data/ssq_only_test.csv', index=False)